<a href="https://colab.research.google.com/github/iamchenchu/Deep-Learnig-with-TensorFlow/blob/main/09_Natural_Language_Processing_with_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP(Naturala Language Processing) and NLU (Natural Language Understanding) :

`A handful of example natural language processing (NLP) and natural language understanding (NLU) problems. These are also often referred to as sequence problems (going from one sequence to another)`

NLP has the goal of deriving information out of natural language (Could be sequences text or speech). Another common term of NLP probelems is sequence to sequence problems (seq2seq)

**Let's see where do we use it : Which are reffered to as a Sequence Problems**

1. MultiLabel Classification
2. Machine Translation
3. Text Generation
4. Voice Assistant

**Types of Sequence problems :(input to output types)**
1. One to one  (one word translation)
2. One to many (Image captioning, possible to have multiple captions)
3. Many to one (Sentiment analysis maybe a youtube comment, BitCoin Price Prediction)
4. many to many (Part-of-speech tagging in natural language processing. Each word in the input sentence may correspond to multiple possible part-of-speech tags.)
5. many to many (DNA sequence alignment. Aligning two DNA sequences where each base in one sequence may correspond to multiple bases in the other.)







# What will we do in this section :         

* Downloading and preparing the text data
* How to prepare text data for modeling (Tokenization and embedding)
* Setting up multiple modeling experiments with Recurrent Neural Networks (RNN)
* Building a text feature extraction model using TensorFlow Hub
* Finding the most wrong prediction examples
* Using a model we've built to make predictions on the text from the wild



**NLP Inputs and outputs :**

`Text -> turn into numbers -> build a model -> train the model to find patterns -> use patterns (make predictions)`

esource: For a great overview of NLP and the different problems within it, read the article A Simple Introduction to [Natural Language Processing](https://becominghuman.ai/a-simple-introduction-to-natural-language-processing-ea66a1747b32).



# STEPS  WE FOLLOW IN CREATING IN RNN / CNN FOR NLP

1. Get the data ready (Turn all data into numbers as neural networks can't handle text or natural language and make sure all of your tensors are in right shape pad sequence which don't fit )
2. Build or pick a pretrained model (to suit your problem)
3. Fit the model to the data and make a prediction
4. Evaluate the model
5. Improve through Experimentation
6. Save and reload your trained model



# TYPICAL ARCHITECTURE OF A RECCURENT NEURAL NETWORK

**What is RNN :**  is class of Artificial Neural Networks where connections between nodes from a directed graph along a temporal sequence. This allows it to exhibit temporal dynamic behaviour.

1. Input Text
2. Input Layers
3. Text vectorization layer
4. Embedding (Turns mapping of text vectors to embedding matrix (representing how words relate Ex King-man+woman = queen))
5. RNN Cells (Finds Pattern in sequence) `Ex : Simple RNN, LSTM, GRU`
6. Hidden Activations : Adds non linearity to learned features usually use `tanh`
7. Pooling layer : Reduces the dimensionality of the learned features usually Conv1D models `Ex : GlobalAveragePooling1D or GlobalMaxPool1D`
8. Fully Connected layer : Further refines learned features from recurrent layers (Dense layer)
9. Output layer (Takes learned feature and outputs them in shape of target labels) Ex : `output_shape = [number_of_classes] (Disaster or not)`
10. Output Activation : Adds non-linearities to output layer.






# Check GPU

In [1]:
!nvidia-smi

Wed Jan  3 15:51:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Get helper functions

In [2]:
# Get helper functions Import the series of the helper functions using the below link

!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

#Import the functions using import
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2024-01-03 15:51:30--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-01-03 15:51:30 (66.5 MB/s) - ‘helper_functions.py’ saved [10246/10246]



# Get the Text Dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of tweets labelled as diaster or not diaster)

See the original source here :https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip



In [3]:
#Get the data

!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

#Unzip the data
unzip_data("nlp_getting_started.zip")

--2024-01-03 15:51:34--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.218.207, 108.177.11.207, 108.177.12.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.218.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.008s  

2024-01-03 15:51:34 (75.7 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



# Becoming one with the data and visualizing a text dataset

To visualize our text samples, we first have to read them in, one way to do so would be to use python: https://realpython.com/python-csv/#:~:text=Reading%20from%20a%20CSV%20file,which%20does%20the%20heavy%20lifting.


 but I prefer to get visual straight way

 So another way to do this to use pandas.



In [4]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df["text"][10]

'Three people died from the heat wave so far'

In [6]:
#Shuffle training dataframe
train_df_shuffled = train_df.sample(frac =1, random_state = 42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
len(train_df), len(test_df)

(7613, 3263)

In [8]:
#What does the test dataframe look like

test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
#How many total samples
len(train_df) + len(test_df)

10876

In [10]:
#Let's visualize some random training examples
import random

random_index = random.randint(0, len(train_df)-5) # create a random indexes not higher than the total num
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target : {target}", "(real diaster)" if target >0 else "(not diaster)")
  print(f"Text :  {text}\n")
  print("---\n")

Target : 1 (real diaster)
Text :  Courageous and honest analysis of need to use Atomic Bomb in 1945. #Hiroshima70 Japanese military refused surrender. https://t.co/VhmtyTptGR

---

Target : 0 (not diaster)
Text :  @ZachZaidman @670TheScore wld b a shame if that golf cart became engulfed in flames. #boycottBears

---

Target : 1 (real diaster)
Text :  Tell @BarackObama to rescind medals of 'honor' given to US soldiers at the Massacre of Wounded Knee. SIGN NOW &amp; RT! https://t.co/u4r8dRiuAc

---

Target : 1 (real diaster)
Text :  Worried about how the CA drought might affect you? Extreme Weather: Does it Dampen Our Economy? http://t.co/fDzzuMyW8i

---

Target : 0 (not diaster)
Text :  @YoungHeroesID Lava Blast &amp; Power Red #PantherAttack @JamilAzzaini @alifaditha

---



# Split data into training and validation sets



In [11]:
from sklearn.model_selection import train_test_split



In [12]:
# Use train_test_split to split training data into training and validation sets

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size = 0.1, # use 10% of training data for validation
                                                                            random_state = 42)

In [13]:
#Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [14]:
#Check the first 10 samples

train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

# Converting text into numbers

When dealing with the text problems, one of the first things you'll have to do before you can build a model is to convert text to numbers.

`There are few ways to do this, namely :`  
**Tokenization :** Straight mapping from token to number (Can be modelled but quickly get's too big)

**Ebedding :** Richer representation of relationships between tokens (Can limit size + can be learned) : Create a matrix of feature vector for each token (the size of the feature vector can be defined and this embeding can be learned)





# Text Vectorization (Tokenization)

In [15]:
train_sentences[:5]


array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [16]:
# Text Vectorization (Tokenization)

import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

#Use the default TextVectorization
text_vectorizer = TextVectorization(max_tokens=10000, # How many words in the vocabulary (automatically add  <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams = None, # Create groups of n-words
                                    output_mode = "int", # how to map tokens to numbers
                                    output_sequence_length = None, #how long do you want your sequences to be
                                    pad_to_max_tokens = True)

In [17]:
len(train_sentences[0].split())

7

In [18]:
#Find the average number of tokens (words) in the training tweets

round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [19]:
#Setup text vectorization variables
max_vocab_length = 10000 # Max number of words to have in our vocabulary
max_length = 15 # Max length our sequences will be

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length)

In [20]:
#Fit the text vectorizer to the training text

text_vectorizer.adapt(train_sentences)


In [21]:
train_sentences[:10]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
      dtype=object)

In [22]:
#Create a sample sentence and tokenize it
sample_sentance = "There is a flood in my street"
text_vectorizer([sample_sentance])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [23]:
# Choose a random sentence from a training dataset and tokenize it

random_sentence = random.choice(train_sentences)
print(f"Original text : {random_sentence}\
        \n\nVectorized version :")
text_vectorizer([random_sentence])


Original text : Carolina accident: Motorcyclist Dies in I-540 Crash With Car That Crossed Median: A motorcycle rider traveling... http://t.co/p18lzRlmy6        

Vectorized version :


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1,  128, 1499, 1216,    4,    1,   85,   14,  127,   16, 2506,
           1,    3, 2855, 2740]])>

In [24]:
#Get the unique words in the vocabulary

words_in_vocab = text_vectorizer.get_vocabulary() #get the all of the unique words in the
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocabulary : {len(words_in_vocab)}")
print(f"Top 5 words in vocabulary : {top_5_words}")
print(f"Bottom 5 words in vocabulary : {bottom_5_words}")

Number of words in vocabulary : 10000
Top 5 words in vocabulary : ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 words in vocabulary : ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


# Creating an Embedding using an Embedding Layer

Turns a positive integers (indexes) into dense vectors of fixed size

To make our embeddding,  we're going to use tensorflow layer and for more infor please visit the below link : https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameters we care most about  for our embedding layer :      

* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented 100 long.
* `input_length`  = Length of the sequences being passes to the embedding layer



In [25]:
from tensorflow.keras import layers

#Create a embedding layer
embedding = layers.Embedding(input_dim = max_vocab_length, # Set input shape
                             output_dim = 128, # Output shape
                             input_length = max_length) # How long is each input
embedding

In [26]:
#Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original Text : {random_sentence}\
      \n \n Embedded Version :" )

#Embed the random sentences (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original Text : Ebay Snipe RT? http://t.co/SlQnph34Nt Lego Power Miners Set 8960 Thunder Driller Boxed. ?Please Favorite &amp; Share      
 
 Embedded Version :


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.04119458, -0.02313271, -0.02204148, ...,  0.03517863,
          0.02698301,  0.04420204],
        [-0.01637862,  0.01413306, -0.03844893, ..., -0.0055123 ,
         -0.00733632,  0.03999866],
        [-0.01923656, -0.00993394,  0.0398787 , ...,  0.01311438,
         -0.04729449, -0.02873572],
        ...,
        [-0.00886745, -0.00163556,  0.0487514 , ...,  0.04596683,
          0.02453313, -0.01665119],
        [-0.00622674, -0.01534403,  0.04969278, ...,  0.04146871,
          0.03745956,  0.02962622],
        [ 0.02202113, -0.01757354, -0.04182756, ..., -0.00688614,
          0.03396118,  0.02971995]]], dtype=float32)>

In [27]:
#Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.04119458, -0.02313271, -0.02204148, -0.02037184, -0.04723653,
        -0.02342043,  0.00747354, -0.02697696, -0.00216382,  0.01938195,
         0.00226667,  0.00875739,  0.02090554, -0.03137138, -0.00125449,
        -0.02551831, -0.0451914 , -0.00301795,  0.01298087,  0.01110848,
        -0.02898088,  0.03583772,  0.03536314,  0.03147218,  0.02229705,
         0.04188993, -0.01784525,  0.04021015, -0.03567393,  0.02773196,
         0.04095193,  0.03341887, -0.03525732,  0.01016688, -0.00460436,
         0.04520531,  0.02787094, -0.04155822,  0.04170981,  0.0187248 ,
         0.00776051, -0.01523728, -0.04509705, -0.00271574,  0.04183037,
         0.04151669, -0.02106861, -0.02195313, -0.03129898, -0.01025786,
        -0.00257146, -0.00190904, -0.03267324, -0.00917893, -0.0305568 ,
        -0.04453871,  0.04594575,  0.0461025 ,  0.02515033, -0.02143547,
        -0.04542563,  0.02009907,  0.04297392,  0.00578958,  0.01241972,
  

## Modeling a text dataset (Running a series of experiments)

Once you've got your inputs and outputs prepared, it's a matter of figuring out which machine learning model to build in between them to bridge the gap.

Now that we've got a way to turn our text data into numbers, we can start to build machine learning models to model it.

To get plenty of practice, we're going to build a series of different models, each as its own experiment. We'll then compare the results of each model and see which one performed best.

More specifically, we'll be building the following:

`Model 0`: Naive Bayes  TF-IDF (baseline) [Sklearn ML Map](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)

`Model 1`: Feed-forward neural network (dense model)

`Model 2`: LSTM model

`Model 3`: GRU model

`Model 4`: Bidirectional-LSTM model

`Model 5`: 1D Convolutional Neural Network

`Model 6`: TensorFlow Hub Pretrained Feature Extractor (Using transfer learning for NLP)

`Model 7`: Same as model 6 with 10% of training data


Model 0 is the simplest to acquire a baseline which we'll expect each other of the other deeper models to beat.

**Each experiment will go through the following steps:**

*   Construct the model
*   Train the model
*   Make predictions with the model
*   Track prediction evaluation metrics for later comparison

      
      
      
      

# Model 0 : Getting a baseline

As with all machine learning modelling experiments, it's importat to create baseline model so you've got a benchmark for future experiments to build upon.

To create our baseline, we'll use Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers.

**Note :** It's common practice to use non-DL algorithms as baseline because of their speed and then later using DL to see if you can improve upon them.



In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

#Create tokenization and modeling pipeline
model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()), # Convert words to numbers using tfidf
    ('clf', MultinomialNB()) # model the text
])

#Fit the pipeline to training the data
model_0.fit(train_sentences, train_labels)



Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [29]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our Baseline model achieves an accuracy : {round(baseline_score * 100)}%")

Our Baseline model achieves an accuracy : 79%


In [30]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

# Creating an evaluation function for our experiments

We could evaluate all of our model predictions with different metrics every time however, this would be cumbersome and could

Let's create 1 to compare our model's predictions with the truth labels using the following metrics :     

* Accuracy
* Precision
* Recall
* F-1 Score

For a deep overview of many different evaluation methods, see the sklearn documentation :     https://scikit-learn.org/stable/modules/model_evaluation.html


In [31]:
# Function to evaluate : Accuracy, precision, recall, f1-score

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred) :
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred)*100

  # Calculate model precision, recall, and f1-score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy ": model_accuracy,
                   "precision ": model_precision,
                   "recall ": model_recall,
                   "f1 ": model_f1}

  return model_results








In [32]:
#Get baseline results
baseline_results = calculate_results(y_true = val_labels,
                                     y_pred=baseline_preds)

baseline_results

{'accuracy ': 79.26509186351706,
 'precision ': 0.8111390004213173,
 'recall ': 0.7926509186351706,
 'f1 ': 0.7862189758049549}

# Model 1: A Simple Dense Model (Feed-forward neural network)

In [33]:
# Create a tensorboard callback (Need to create a new one for each model)
from helper_functions import create_tensorboard_callback

#Create a directory to save tensorboard logs
SAVE_DIR = "model_logs"




In [34]:
# Build model with the Functional API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
# x = layers.GlobalMaxPooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

In [35]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [36]:
# Compile model
model_1.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [37]:
# Fit the model
model_1_history = model_1.fit(x = train_sentences,
                              y=train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                       experiment_name = "model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20240103-155138
Epoch 1/5
215/215 [==============================] - 8s 30ms/step - loss: 0.6109 - accuracy: 0.6957 - val_loss: 0.5358 - val_accuracy: 0.7480
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.4407 - accuracy: 0.8200 - val_loss: 0.4690 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3460 - accuracy: 0.8606 - val_loss: 0.4605 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2834 - accuracy: 0.8889 - val_loss: 0.4615 - val_accuracy: 0.7808
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2379 - accuracy: 0.9118 - val_loss: 0.4815 - val_accuracy: 0.7861


In [38]:
#Check the results

model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4815 - accuracy: 0.7861


[0.48149019479751587, 0.7860892415046692]

In [39]:
#Make some predictions
model_1_pre_probs = model_1.predict(val_sentences)
model_1_pre_probs.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [40]:
embedding.weights

[<tf.Variable 'embedding/embeddings:0' shape=(10000, 128) dtype=float32, numpy=
 array([[-0.01142132, -0.02175345, -0.04075807, ..., -0.01066437,
          0.015127  , -0.01555117],
        [-0.00198   ,  0.00333528, -0.05006625, ...,  0.00329983,
          0.01391982, -0.024185  ],
        [-0.00258985,  0.02951235, -0.04640581, ...,  0.00994155,
          0.03238501,  0.03415835],
        ...,
        [ 0.04608308, -0.02683973, -0.00901347, ..., -0.04804896,
          0.00924534,  0.03566431],
        [ 0.00294749, -0.06641594, -0.03430351, ..., -0.00598262,
          0.0165428 ,  0.07087287],
        [-0.07112099, -0.10102984, -0.03359052, ..., -0.10404312,
          0.10581555,  0.0705013 ]], dtype=float32)>]

In [41]:
# Make predictions (these come back in the form of probabilities)
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10] # only print out the first 10 prediction probabilities

24/24 [==============================] - 0s 2ms/step


array([[0.29903322],
       [0.83275884],
       [0.99766314],
       [0.10335255],
       [0.09613951],
       [0.92940587],
       [0.92059755],
       [0.9931219 ],
       [0.96818507],
       [0.24452978]], dtype=float32)

In [42]:
val_labels

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,

In [43]:
#COnvert model prediction probabilities to label formats
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)>

In [44]:
#Calculate our model_1 results
model_1_results = calculate_results(y_true = val_labels,
                                    y_pred = model_1_preds)
model_1_results


{'accuracy ': 78.60892388451444,
 'precision ': 0.7925823788155513,
 'recall ': 0.7860892388451444,
 'f1 ': 0.7825463664667974}

In [45]:
baseline_results

{'accuracy ': 79.26509186351706,
 'precision ': 0.8111390004213173,
 'recall ': 0.7926509186351706,
 'f1 ': 0.7862189758049549}

In [46]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

# Visualizing learned embeddings

In [47]:
#Get the vocabulary from the text vectorizatiion layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [48]:
# Model 1 summary

model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [49]:
# Get the weight matrix of embeding layer
# These are the neumarical representations of each token in our training data, which have been learned for 5 epochs

embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights
print(embed_weights.shape) # Same size as vocab size and ebedding_dim (Output dim of our embedding layer)

(10000, 128)


Now we have got the embedding matrix our model has learned to represent our tokens, let's see how we can visualize it.

To do so, Tensorflow has a handy tool called projector : https://projector.tensorflow.org/

And Tensorflow also has an incredible guide on word embeddings themselves :
https://www.tensorflow.org/text/guide/word_embeddings


In [50]:
embed_weights

array([[-0.01142132, -0.02175345, -0.04075807, ..., -0.01066437,
         0.015127  , -0.01555117],
       [-0.00198   ,  0.00333528, -0.05006625, ...,  0.00329983,
         0.01391982, -0.024185  ],
       [-0.00258985,  0.02951235, -0.04640581, ...,  0.00994155,
         0.03238501,  0.03415835],
       ...,
       [ 0.04608308, -0.02683973, -0.00901347, ..., -0.04804896,
         0.00924534,  0.03566431],
       [ 0.00294749, -0.06641594, -0.03430351, ..., -0.00598262,
         0.0165428 ,  0.07087287],
       [-0.07112099, -0.10102984, -0.03359052, ..., -0.10404312,
         0.10581555,  0.0705013 ]], dtype=float32)

In [51]:
#Create embedding files (We got this from tensorFlow's word embeddings documentation)

"""import io

out_v = io.open('vectors.tsv', 'w', encoding = 'utf-8')
out_m = io.open('metadata.tsv', 'w', encoding = 'utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue # Skip 0, it's padding
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()"""

'import io\n\nout_v = io.open(\'vectors.tsv\', \'w\', encoding = \'utf-8\')\nout_m = io.open(\'metadata.tsv\', \'w\', encoding = \'utf-8\')\n\nfor index, word in enumerate(words_in_vocab):\n  if index == 0:\n    continue # Skip 0, it\'s padding\n  vec = embed_weights[index]\n  out_v.write(\'\t\'.join([str(x) for x in vec]) + "\n")\n  out_m.write(word + "\n")\nout_v.close()\nout_m.close()'

Reseourses : Word2Vec https://jalammar.github.io/illustrated-word2vec/


In [52]:
#Download files from Colab to upload to projector
"""
try :
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception as e:
  pass"""

"\ntry :\n  from google.colab import files\n  files.download('vectors.tsv')\n  files.download('metadata.tsv')\nexcept Exception as e:\n  pass"

Once you've downloaded the embedding vectors and metadata, you can visualize them using Embedding Vector tool:

Go to http://projector.tensorflow.org/
Click on "Load data"
Upload the two files you downloaded (embedding_vectors.tsv and embedding_metadata.tsv)
Explore
Optional: You can share the data you've created by clicking "Publish"
What do you find?

Are words with similar meanings close together?

Remember, they might not be. The embeddings we downloaded are how our model interprets words, not necessarily how we interpret them.

Also, since the embedding has been learned purely from Tweets, it may contain some strange values as Tweets are a very unique style of natural language.

🤔 Question: Do you have to visualize embeddings every time?

No. Although helpful for gaining an intuition of what natural language embeddings are, it's not completely necessary. Especially as the dimensions of your vocabulary and embeddings grow, trying to comprehend them would become an increasingly difficult task.

# Recurrent Neural Networks (RNN's) :     

RNN's are useful for sequence of data.

The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of a later input.

**If you want an overview of the internals of a recurrent neural network, see the following :**     
  * MIT Sequence modeling lecture : https://www.youtube.com/watch?v=ySEx_Bqxvvo
  * Chris Olah's intro to LSTMs : https://colah.github.io/posts/2015-08-Understanding-LSTMs/
  * Andre Karpathy's the unreasonable effectiveness of recurrent neural networks : https://karpathy.github.io/2015/05/21/rnn-effectiveness/





# Model. 2: LSTM [Long Short Term Memory] :

## One of the most popular LSTM Cells :

Our structure of an RNN typically looks like this:      



```
Input(text) -> Tokenize -> Embedding -> Layers(RNNs/Debse) -> Output(Label probability)

```







In [53]:
#Create an LSTM Model

from tensorflow.keras.layers import LSTM
inputs = layers.Input(shape=(1,), dtype ="string")
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
# x = layers.LSTM(units = 64, return_sequences=True)(x) # It would be better if it's multiple of 8, when you are stacking RNN cells together u need to return sequences = True
# print(x.shape)
x = layers.LSTM(64)(x)
# print(x.shape)
# x = layers.Dense(64, activation="relu")(x)
# print(x.shape)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [54]:
# Get a summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1329473 (5.07 MB)
Trainable params: 1329473 (5.07 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [55]:
# Compile the model
model_2.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])



In [56]:
#Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20240103-155153
Epoch 1/5
215/215 [==============================] - 9s 33ms/step - loss: 0.2169 - accuracy: 0.9216 - val_loss: 0.5790 - val_accuracy: 0.7887
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1565 - accuracy: 0.9396 - val_loss: 0.6050 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1276 - accuracy: 0.9499 - val_loss: 0.6537 - val_accuracy: 0.7848
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1060 - accuracy: 0.9565 - val_loss: 0.7515 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0854 - accuracy: 0.9661 - val_loss: 0.9021 - val_accuracy: 0.7782


In [57]:
#Make predictions with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]


24/24 [==============================] - 0s 2ms/step


array([[4.53598704e-03],
       [8.61206412e-01],
       [9.99696255e-01],
       [1.02565035e-01],
       [4.62481898e-04],
       [9.96359527e-01],
       [7.25314200e-01],
       [9.99794662e-01],
       [9.99603450e-01],
       [8.57909501e-01]], dtype=float32)

In [58]:
# Convert model 2 pred probs to labels

model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [68]:
#Calculate Model 2 results
model_2_results = calculate_results(y_true = val_labels,
                                    y_pred = model_2_preds)
model_2_results

{'accuracy ': 77.82152230971128,
 'precision ': 0.7804499491198106,
 'recall ': 0.7782152230971129,
 'f1 ': 0.7760126933653841}

In [60]:
baseline_results

{'accuracy ': 79.26509186351706,
 'precision ': 0.8111390004213173,
 'recall ': 0.7926509186351706,
 'f1 ': 0.7862189758049549}

# Model 3 : GRU

Another popular and effective RNN component is the GRU or gated recurrent unit.

the GRU cell has similar features ato an LSTM cell but has less parameters.

Resource : https://towardsdatascience.com/understanding-gru-networks-2ef37df6c9be

In [75]:
#Build an RNN using the GRU cell

from tensorflow.keras.layers import GRU
inputs = layers.Input(shape=(1, ), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
# print(x.shape)
# x = layers.GRU(64, return_sequences =True)(x) # If you want to stack recurrent layers on top of each other you need return_sequences = True
# print(x.shape)
# x = layers.LSTM(42, return_sequences =True)(x)
# print(x.shape)
# x = layers.GRU(99)(x)
# print(x.shape)
# x = layers.Dense(64, activation = "relu")(x)
# x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation ="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [76]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru_7 (GRU)                 (None, 64)                37248     
                                                                 
 dense_9 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1317313 (5.03 MB)
Trainable params: 1317313 (5.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________

In [77]:
# Compile the model
model_3.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [78]:
# Fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data =(val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR, "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20240103-162929
Epoch 1/5
215/215 [==============================] - 9s 32ms/step - loss: 0.1625 - accuracy: 0.9367 - val_loss: 0.7905 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0858 - accuracy: 0.9679 - val_loss: 0.7928 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 2s 7ms/step - loss: 0.0724 - accuracy: 0.9720 - val_loss: 0.8083 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 2s 7ms/step - loss: 0.0611 - accuracy: 0.9746 - val_loss: 1.2664 - val_accuracy: 0.7664
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0527 - accuracy: 0.9777 - val_loss: 1.0194 - val_accuracy: 0.7808


In [79]:
#Make some predictions with GRU model
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[5.1160175e-03],
       [7.3305917e-01],
       [9.9976224e-01],
       [9.4641492e-02],
       [1.5062930e-04],
       [9.9957150e-01],
       [9.5090550e-01],
       [9.9991727e-01],
       [9.9983239e-01],
       [7.2318327e-01]], dtype=float32)

In [80]:
# Convert model 3 pred probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [81]:
# Calculate model 3 results
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred = model_3_preds)
model_3_results

{'accuracy ': 78.08398950131233,
 'precision ': 0.7834583238190046,
 'recall ': 0.7808398950131233,
 'f1 ': 0.778533312750939}

#Model 4 :  Bidirectional RNN Model

Resource : https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional

Normal RNN's go from left to right (just like you'd read an English sentence) however, a birectional RNN goes from right to left as well as left to right.




In [86]:
#Build Birectional RNN in tensorflow
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional")


In [87]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional_3 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dense_11 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1378945 (5.26 MB)
Trainable par

In [88]:
#Compile the model
model_4.compile(loss="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [89]:
# Fit the model
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs =5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                       "model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20240103-164824
Epoch 1/5
215/215 [==============================] - 12s 41ms/step - loss: 0.1079 - accuracy: 0.9676 - val_loss: 1.0953 - val_accuracy: 0.7717
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0541 - accuracy: 0.9766 - val_loss: 1.1517 - val_accuracy: 0.7717
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0477 - accuracy: 0.9783 - val_loss: 1.4001 - val_accuracy: 0.7703
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0447 - accuracy: 0.9797 - val_loss: 1.4493 - val_accuracy: 0.7585
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0396 - accuracy: 0.9803 - val_loss: 1.4642 - val_accuracy: 0.7664


In [99]:
#Make predictions with our model
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[1.5216758e-03],
       [7.6126707e-01],
       [9.9999094e-01],
       [2.3801343e-01],
       [2.0996127e-05],
       [9.9988806e-01],
       [9.7036809e-01],
       [9.9999571e-01],
       [9.9999332e-01],
       [9.9824810e-01]], dtype=float32)

In [101]:
# Convert pred probs to pred labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [103]:
#Convert pred probs to pred labels
model_4_results = calculate_results(y_true = val_labels,
                                  y_pred = model_4_preds)
model_4_results

{'accuracy ': 76.64041994750657,
 'precision ': 0.7674517658976838,
 'recall ': 0.7664041994750657,
 'f1 ': 0.7645419362124183}

In [104]:
model_3_results

{'accuracy ': 78.08398950131233,
 'precision ': 0.7834583238190046,
 'recall ': 0.7808398950131233,
 'f1 ': 0.778533312750939}

# Convolutional Neural Networks for Text (and other types of sequences)

We've used CNNs for images but images are typically 2D (height * width)... however out text is 1D

previouslu we've Conv2D for our image data but now we're going to use Conv1D for text.

The Trypical structure of a Conv1D model for sequence (in our case, text):
I

```
# The Trypical structure of a Conv1D model for sequence (in our case, text):
Inputs(Text) -> Tokanization -> Embedding -> Layer(s) -> (Typically Conv1D + Pooling) -> output (class probabilities)
```

# Model 5 : Conv 1D

For different explanations of parameters see : https://stackoverflow.com/questions/37674306/what-is-the-difference-between-same-and-valid-padding-in-tf-nn-max-pool-of-t

* CNN explainer for 2D and 1D : https://poloclub.github.io/cnn-explainer/


In [107]:
# Test out our embedding layer, Conv1D layer and max pooling
from tensorflow.keras import layers

embedding_test = embedding(text_vectorizer(["this is a test sentence"])) # TUrns target sequence into embedding
conv_1d = layers.Conv1D(filters = 32, # Hidden units
                        kernel_size = 5,  # this is also reffered to ngram og 5 (meaning it looks at 5 words at a time)
                        strides =1,  # Default
                        activation ="relu",
                        padding ="valid") # Default = "valid", the output is smaller than input shape, "same" means output is same
conv_1d_output = conv_1d(embedding_test) # pass test embedding through conv1d layer
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))